Credit where credit is due: this is based on  https://www.kaggle.com/code/gpreda/fast-test-of-llama-v2-pre-quantized-with-llama-cpp.

Loading the model will take a while so please be patient

In [1]:
# clean up after failed efforts if necessary
!rm -rf llama.cpp
# we need no CUDA for CPU; this flag has changed since the initial release
!CMAKE_ARGS="-DLLAMA_CUDA=off" pip install llama-cpp-python 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.3/50.3 MB 23.7 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... - \ | / - \ | done
  Preparing metadata (pyproject.toml) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.79-cp310-cp310-linux_x86_64.whl size=3663689 sha256=dc5a60c7328a498adc383c2e93116430314d472ac456110b95c6054e57da76fe
  Stored in directory: /root/.cache/pip/wheels/bb/2e/11/8b10c6b698e6abc1289e9919e098ac4bcf6b16ebd46153e8ba
Successfully built llama-cpp-python


In [2]:
import arrow
from llama_cpp import Llama

time_load = arrow.now()
llm = Llama(
    model_path="/kaggle/input/meta-llama-3-8b-gguf/llama3-8b-instruct-Q5_K_M.gguf",
    n_ctx=4096,      # Max tokens for in + out
    n_threads=4,     # CPU cores used
    n_gpu_layers=-1,  # Load all layers into VRAM of the GPU
)
print("Model load time: {:5.4f} seconds".format((arrow.now() - time_load).total_seconds()))

llama_model_loader: loaded meta data with 22 key-value pairs and 291 tensors from /kaggle/input/meta-llama-3-8b-gguf/llama3-8b-instruct-Q5_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = .
llama_model_loader: - kv   2:                           llama.vocab_size u32              = 128256
llama_model_loader: - kv   3:                       llama.context_length u32              = 8192
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   7:                 lla

Model load time: 44.2420 seconds


AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | AVX512_BF16 = 0 | FMA = 1 | NEON = 0 | SVE = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | MATMUL_INT8 = 0 | LLAMAFILE = 1 | 
Model metadata: {'tokenizer.chat_template': "{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}", 'tokenizer.ggml.eos_token_id': '128009', 'general.quantization_version': '2', 'tokenizer.ggml.model': 'gpt2', 'general.architecture': 'llama', 'llama.rope.freq_base': '500000.000000', 'llama.context_length': '8192', 'general.name': '.', 'llama.vocab_size': '128256', 'general.file_type': '17', 'llama.embedding_length': '4096', 'lla

In [3]:
def ask(query: str, max_tokens: int):
    time_ask = arrow.now()
    output = llm(prompt="Q: {} A: ".format(query), max_tokens=max_tokens, stop=["Q:", "\n"], echo=False)
    return [output['choices'][index]['text'] for index in range(len(output['choices']))], (arrow.now() - time_ask).total_seconds()

query_cities = 'What are the names of three European capital cities?'
result_cities, time_cities = ask(query=query_cities, max_tokens=40)
print(query_cities)
print(result_cities)
print('time: {:5.4f}'.format(time_cities))


llama_print_timings:        load time =    4698.01 ms
llama_print_timings:      sample time =      44.10 ms /    20 runs   (    2.21 ms per token,   453.48 tokens per second)
llama_print_timings: prompt eval time =    4697.77 ms /    15 tokens (  313.18 ms per token,     3.19 tokens per second)
llama_print_timings:        eval time =    6996.55 ms /    19 runs   (  368.24 ms per token,     2.72 tokens per second)
llama_print_timings:       total time =   11770.51 ms /    34 tokens


What are the names of three European capital cities?
['1. Paris (France), 2. Rome (Italy), 3. Berlin (Germany)']
time: 11.7883


In [4]:
query_catcher = 'What can you tell me about the J. D. Salinger novel The Catcher in the Rye?'
result_catcher, time_catcher = ask(query=query_catcher, max_tokens=1000)
print(query_catcher)
print(result_catcher)
print('time: {:5.4f}'.format(time_catcher))

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4698.01 ms
llama_print_timings:      sample time =     480.92 ms /   197 runs   (    2.44 ms per token,   409.63 tokens per second)
llama_print_timings: prompt eval time =    6001.41 ms /    24 tokens (  250.06 ms per token,     4.00 tokens per second)
llama_print_timings:        eval time =   74400.58 ms /   196 runs   (  379.59 ms per token,     2.63 tokens per second)
llama_print_timings:       total time =   81217.53 ms /   220 tokens


What can you tell me about the J. D. Salinger novel The Catcher in the Rye?
['1) The Catcher in the Rye is a coming-of-age story written by J.D. Salinger, published in 1951. It follows the journey of Holden Caulfield, a 16-year-old boy who struggles to find his place in society after being expelled from school. 2) The novel explores themes such as alienation, rebellion, and the challenges of adolescence, which are still relevant today. 3) The title "The Catcher in the Rye" is often misinterpreted; it refers to Holden\'s desire to protect children from falling off a cliff, symbolizing his concern for the innocence and vulnerability of young people. 4) The novel has become a classic of American literature, with its protagonist, Holden Caulfield, becoming an iconic figure in Western cultural consciousness. 5) The Catcher in the Rye is known for its simple yet powerful prose, which captures the voice and perspective of a rebellious teenager struggling to find his identity.']
time: 81.2258


In [5]:
r, t = ask(query='How do I drive from Washington DC to New York City?', max_tokens=3000)
print(r)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4698.01 ms
llama_print_timings:      sample time =      38.07 ms /    17 runs   (    2.24 ms per token,   446.53 tokens per second)
llama_print_timings: prompt eval time =    4703.26 ms /    15 tokens (  313.55 ms per token,     3.19 tokens per second)
llama_print_timings:        eval time =    5772.11 ms /    16 runs   (  360.76 ms per token,     2.77 tokens per second)
llama_print_timings:       total time =   10537.00 ms /    31 tokens


['4 hours and 30 minutes by car, or take the train. (more...)']


In [6]:
r0, t0 = ask(query='What would I need to do to build an edible bicycle?', max_tokens=3000)
print(r0)

Llama.generate: prefix-match hit

llama_print_timings:        load time =    4698.01 ms
llama_print_timings:      sample time =     121.37 ms /    54 runs   (    2.25 ms per token,   444.91 tokens per second)
llama_print_timings: prompt eval time =    3800.92 ms /    15 tokens (  253.39 ms per token,     3.95 tokens per second)
llama_print_timings:        eval time =   19481.20 ms /    53 runs   (  367.57 ms per token,     2.72 tokens per second)
llama_print_timings:       total time =   23478.05 ms /    68 tokens


[' While the idea of building an edible bicycle might be intriguing, it is important to note that bicycles are typically made of materials such as metal, plastic, and other synthetic substances. These materials are not edible and cannot be used to create a functional bicycle that can be consumed.']
